# Extract Neighborhoods (Nbh) Overviews
----

#### Note
* Dataset request from rapidApi/Mashvisor endpoint top neighborhoods within top cities
* Dataset request from rapidApi/Mashvisor endpoint neighborhood overview

In [3]:
# Interactive plots
%matplotlib notebook

# Dependencies and Setup
import requests
import random
import json
import ast
import os
import datetime
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import scipy.stats as st
#from citipy import citipy
#from pprint import pprint

# Set by selecting a datasource import type (CSV or API)
source = 'api'
#source = 'csv'

# Remember to update the config file with your API key
from api_keys import mash_api_key

# Set variable for current date using datetime library
today = datetime.date.today()

# Set Mashvisor API URL for rental rates
urlTop = "https://mashvisor-api.p.rapidapi.com/trends/neighborhoods"
urlOverview = "https://mashvisor-api.p.rapidapi.com/neighborhood/"

# Set API variables
states = ["TX","CA","NY","FL","DC"]

# Set todays date as active and formatted as mm/dd/yy for API import
date = today.strftime('%m/%d/%y')
datef = today.strftime('%m%d%y')

# Set a specific date as active parameter for CSV import
#date = '09/12/20'
#datef = '091220'

# Print active date
print(date)
#print(mash_api_key)

12/03/20


In [12]:
f = open(f'../data/mash-api/{datef}/topAirbnbCities-{datef}j.json', 'r')
s = f.read()
s = s.replace("\'", "\"")
s = s.replace("null", "''")
nbhstr = json.dumps(s)
nbhjson = json.loads(nbhstr)
nbhbnb = ast.literal_eval(nbhjson)
df_nbh = pd.DataFrame(nbhbnb)
df_nbh = df_nbh.T
df_nbh

,state,occupancy,total_listing
San Antonio,TX,56.797,8140
Austin,TX,52.7838,4919
Fort Worth,TX,55.8932,3326
Dallas,TX,55.943,2748
Houston,TX,52.8638,2638
Oakland,CA,63.639,6979
Los Angeles,CA,66.4794,5552
San Francisco,CA,60.9809,5069
San Diego,CA,61.358,4513
Long Beach,CA,63.2271,4043


In [22]:
# Create and call Mashvisor API query string for top 5 neighborhoods within top cities
responseTop = []
for i in range(len(df_nbh)):
    querystring = {"city":df_nbh.index[i],"state":df_nbh.iloc[i][0],"page":"1","items":"5"}
    print(f'extracting {df_nbh.index[i]},{df_nbh.iloc[i][0]} top neighborhoods...')
    headers = {
        'x-rapidapi-key': mash_api_key,
        'x-rapidapi-host': "mashvisor-api.p.rapidapi.com"
    }        
  
    response = requests.request("GET", urlTop, headers=headers, params=querystring)
    responseTop.append(response.json())
    print('data merged')
    print('')

extracting San Antonio,TX top neighborhoods...
data merged

extracting Austin,TX top neighborhoods...
data merged

extracting Fort Worth,TX top neighborhoods...
data merged

extracting Dallas,TX top neighborhoods...
data merged

extracting Houston,TX top neighborhoods...
data merged

extracting Oakland,CA top neighborhoods...
data merged

extracting Los Angeles,CA top neighborhoods...
data merged

extracting San Francisco,CA top neighborhoods...
data merged

extracting San Diego,CA top neighborhoods...
data merged

extracting Long Beach,CA top neighborhoods...
data merged

extracting Fort Lauderdale,FL top neighborhoods...
data merged

extracting Jacksonville,FL top neighborhoods...
data merged

extracting Tampa,FL top neighborhoods...
data merged

extracting Pensacola,FL top neighborhoods...
data merged

extracting Saint Petersburg,FL top neighborhoods...
data merged

extracting New York,NY top neighborhoods...
data merged

extracting Buffalo,NY top neighborhoods...
data merged

extra

In [28]:
print(len(responseTop))
df_top = pd.DataFrame(responseTop)
df_top.head()

21


,status,content,message
0,success,"{'total_results': 5, 'current_page': '1', 'sta...",City Overview fetched successfully
1,success,"{'total_results': 5, 'current_page': '1', 'sta...",City Overview fetched successfully
2,success,"{'total_results': 5, 'current_page': '1', 'sta...",City Overview fetched successfully
3,success,"{'total_results': 5, 'current_page': '1', 'sta...",City Overview fetched successfully
4,success,"{'total_results': 5, 'current_page': '1', 'sta...",City Overview fetched successfully


In [29]:
result = df_top.to_json(orient="records")
parsed = json.loads(result)
top_json = json.dumps(parsed, indent=4)
print(top_json)

[
    {
        "status": "success",
        "content": {
            "total_results": 5,
            "current_page": "1",
            "state": "TX",
            "city": "San Antonio",
            "neighborhoods": [
                {
                    "id": 273471,
                    "name": "Dignowity Hill-St. Paul Square",
                    "city": "San Antonio",
                    "state": "TX",
                    "occupancy": 51.6376,
                    "total_listing": 1,
                    "airbnb_listings": 229,
                    "description": null,
                    "single_home_value": 262500,
                    "single_home_value_formatted": "$262,500",
                    "investment_rentals": {
                        "airbnb_rental": {
                            "roi": -1.6581143141,
                            "cap_rate": 10.906461126,
                            "rental_income": 2385.7883713133
                        },
                        "tradition

In [31]:
with open(f'../data/mash-api/{datef}/top_nbh-{datef}.json', 'w') as f1:
    json.dump(top_json,f1)
    print("The top neighborhood json file is created")
f1.close()

The top neighborhood json file is created


In [35]:
topNbhId = []
for i in range(len(df_top)):
    for j in range(5):
        
        nbh = {
            "id":df_top.iloc[i]['content']['neighborhoods'][j]['id'],
            "name":df_top.iloc[i]['content']['neighborhoods'][j]['name'],
            "city":df_top.iloc[i]['content']['neighborhoods'][j]['city'],
            "state":df_top.iloc[i]['content']['neighborhoods'][j]['state']
        }
        topNbhId.append(nbh)
        
df_topNbhId = pd.DataFrame(topNbhId)
df_topNbhId.index = df_topNbhId['id']
del df_topNbhId['id']
df_topNbhId

,name,city,state
id,,,
273471,Dignowity Hill-St. Paul Square,San Antonio,TX
271352,Downtown,San Antonio,TX
275804,United Homeowners,San Antonio,TX
271349,Dignowity Hill,San Antonio,TX
274033,Historic Gardens,San Antonio,TX
...,...,...,...
403138,Lady Bird Johnson Park,Washington,DC
403491,Sursum Corda Cooperative,Washington,DC
121750,Ledroit Park,Washington,DC


In [37]:
result = df_topNbhId.to_json(orient="index")
parsed = json.loads(result)
topNbhId_json = json.dumps(parsed, indent=4)
print(topNbhId_json)

{
    "273471": {
        "name": "Dignowity Hill-St. Paul Square",
        "city": "San Antonio",
        "state": "TX"
    },
    "271352": {
        "name": "Downtown",
        "city": "San Antonio",
        "state": "TX"
    },
    "275804": {
        "name": "United Homeowners",
        "city": "San Antonio",
        "state": "TX"
    },
    "271349": {
        "name": "Dignowity Hill",
        "city": "San Antonio",
        "state": "TX"
    },
    "274033": {
        "name": "Historic Gardens",
        "city": "San Antonio",
        "state": "TX"
    },
    "271360": {
        "name": "East Cesar Chavez",
        "city": "Austin",
        "state": "TX"
    },
    "271303": {
        "name": "Bouldin",
        "city": "Austin",
        "state": "TX"
    },
    "271325": {
        "name": "Central East Austin",
        "city": "Austin",
        "state": "TX"
    },
    "271574": {
        "name": "South River City",
        "city": "Austin",
        "state": "TX"
    },
    "27165

In [40]:
with open(f'../data/mash-api/{datef}/top_nbh_id-{datef}.json', 'w') as f1:
    json.dump(topNbhId_json,f1)
    print("The top neighborhood by id json file is created")
f1.close()

The top neighborhood by id json file is created


In [51]:
# Create and call Mashvisor API query string for OVERVIEW of the top 5 neighborhoods in each city
# Retrieves the Walkingscore
responseOVW = []
for i in range(len(df_topNbhId)):
    urlOVW = urlOverview + str(df_topNbhId.index[i]) + "/bar"
    querystring = {"state":df_topNbhId.iloc[i][2]}
    print(f'extracting {df_topNbhId.index[i]}, {df_topNbhId.iloc[i][0]} neighborhood overview...')
    headers = {
        'x-rapidapi-key': mash_api_key,
        'x-rapidapi-host': "mashvisor-api.p.rapidapi.com"
    }        
  
    response = requests.request("GET", urlOVW, headers=headers, params=querystring)
    responseOVW.append(response.json())
    print('data merged')
    print('')

extracting 273471, Dignowity Hill-St. Paul Square neighborhood overview...
data merged

extracting 271352, Downtown neighborhood overview...
data merged

extracting 275804, United Homeowners neighborhood overview...
data merged

extracting 271349, Dignowity Hill neighborhood overview...
data merged

extracting 274033, Historic Gardens neighborhood overview...
data merged

extracting 271360, East Cesar Chavez neighborhood overview...
data merged

extracting 271303, Bouldin neighborhood overview...
data merged

extracting 271325, Central East Austin neighborhood overview...
data merged

extracting 271574, South River City neighborhood overview...
data merged

extracting 271653, Windsor Road neighborhood overview...
data merged

extracting 422770, Carter Park neighborhood overview...
data merged

extracting 233172, Fairmount neighborhood overview...
data merged

extracting 422844, North Hi Mount neighborhood overview...
data merged

extracting 422756, Berkeley Place neighborhood overview.

In [53]:
print(len(responseOVW))
df_NbhOVW = pd.DataFrame(responseOVW)
df_NbhOVW.head()

105


,status,content,message
0,success,"{'id': '273471', 'name': 'Dignowity Hill-St. P...",Neighborhood bar fetched successfully
1,success,"{'id': '271352', 'name': 'Downtown', 'city': '...",Neighborhood bar fetched successfully
2,success,"{'id': '275804', 'name': 'United Homeowners', ...",Neighborhood bar fetched successfully
3,success,"{'id': '271349', 'name': 'Dignowity Hill', 'ci...",Neighborhood bar fetched successfully
4,success,"{'id': '274033', 'name': 'Historic Gardens', '...",Neighborhood bar fetched successfully


In [54]:
result = df_NbhOVW.to_json(orient="records")
parsed = json.loads(result)
nbhovw_json = json.dumps(parsed, indent=4)
print(nbhovw_json)

[
    {
        "status": "success",
        "content": {
            "id": "273471",
            "name": "Dignowity Hill-St. Paul Square",
            "city": "San Antonio",
            "county": "Bexar",
            "state": "TX",
            "is_village": 0,
            "description": null,
            "image": "https://9ac82074a42d61a93c2a-4910baab8d3df1a59178432e0b86512c.ssl.cf5.rackcdn.com",
            "latitude": 29.42216,
            "longitude": -98.479162,
            "walkscore": 57,
            "num_of_properties": 1,
            "num_of_airbnb_properties": 229,
            "num_of_traditional_properties": 25,
            "median_price": 262500,
            "price_per_sqft": 246.25,
            "mashMeter": 65.89,
            "mashMeterStars": 3,
            "avg_occupancy": 51.6376,
            "strategy": "airbnb",
            "airbnb_rental": {
                "roi": -1.6581143141,
                "cap_rate": 8.3748382813,
                "rental_income": 1831.995874023

In [56]:
with open(f'../data/mash-api/{datef}/top_nbh_overview-{datef}.json', 'w') as f1:
    json.dump(nbhovw_json,f1,indent=4)
    print("The top neighborhood overview json file is created")
f1.close()

The top neighborhood overview json file is created


In [69]:
spfOVW = []
for i in range(len(df_NbhOVW)):
    
        if (df_NbhOVW.iloc[i]['content']['walkscore'] is not None):
            walk_score = int(df_NbhOVW.iloc[i]['content']['walkscore'])
        else:
            walk_score = 0
        
        if (df_NbhOVW.iloc[i]['content']['num_of_airbnb_properties'] is not None):
            airbnb_count = int(df_NbhOVW.iloc[i]['content']['num_of_airbnb_properties'])
        else:
            airbnb_count = 0
            
        if (df_NbhOVW.iloc[i]['content']['num_of_traditional_properties'] is not None):
            other_count = int(df_NbhOVW.iloc[i]['content']['num_of_traditional_properties'])
        else:
            other_count = 0
    
        dict_spfOVW = {
            "id":df_NbhOVW.iloc[i]['content']['id'],
            "name":df_NbhOVW.iloc[i]['content']['name'],
            "city":df_NbhOVW.iloc[i]['content']['city'],
            "county":df_NbhOVW.iloc[i]['content']['county'],
            "state":df_NbhOVW.iloc[i]['content']['state'],
            "lat":df_NbhOVW.iloc[i]['content']['latitude'],
            "lon":df_NbhOVW.iloc[i]['content']['longitude'],
            "walkscore":walk_score,
            "airbnb_count":airbnb_count,
            "other_count":other_count,
            "avg_occupancy":df_NbhOVW.iloc[i]['content']['avg_occupancy'],
            "median_price":df_NbhOVW.iloc[i]['content']['median_price'],
            "sqft_price":df_NbhOVW.iloc[i]['content']['price_per_sqft']
        }
        spfOVW.append(dict_spfOVW)
        
df_spfOVW = pd.DataFrame(spfOVW)
df_spfOVW.index = df_spfOVW['id']
del df_spfOVW['id']
df_spfOVW

,name,city,county,state,lat,lon,walkscore,airbnb_count,other_count,avg_occupancy,median_price,sqft_price
id,,,,,,,,,,,,
273471,Dignowity Hill-St. Paul Square,San Antonio,Bexar,TX,29.422160,-98.479162,57,229,25,51.6376,262500,246.25
271352,Downtown,San Antonio,Bexar,TX,29.424127,-98.487186,74,225,6,52.6800,605000,369.23
275804,United Homeowners,San Antonio,Bexar,TX,29.440150,-98.427296,14,210,0,56.5571,131000,113.72
271349,Dignowity Hill,San Antonio,Bexar,TX,29.427419,-98.472334,51,208,3,53.5433,253750,211.30
274033,Historic Gardens,San Antonio,Bexar,TX,29.415926,-98.474659,45,199,8,51.7940,269950,165.95
...,...,...,...,...,...,...,...,...,...,...,...,...
403138,Lady Bird Johnson Park,Washington,District of Columbia,DC,38.880628,-77.054984,9,477,329,68.6897,529000,NaN
403491,Sursum Corda Cooperative,Washington,District of Columbia,DC,38.904914,-77.010625,94,316,13,69.5791,749900,662.17
121750,Ledroit Park,Washington,District of Columbia,DC,38.916935,-77.018105,84,283,7,69.6396,875000,623.58


In [70]:
result = df_spfOVW.to_json(orient="index")
parsed = json.loads(result)
spfOVW_json = json.dumps(parsed, indent=4)
print(spfOVW_json)

{
    "273471": {
        "name": "Dignowity Hill-St. Paul Square",
        "city": "San Antonio",
        "county": "Bexar",
        "state": "TX",
        "lat": 29.42216,
        "lon": -98.479162,
        "walkscore": 57,
        "airbnb_count": 229,
        "other_count": 25,
        "avg_occupancy": 51.6376,
        "median_price": 262500,
        "sqft_price": 246.25
    },
    "271352": {
        "name": "Downtown",
        "city": "San Antonio",
        "county": "Bexar",
        "state": "TX",
        "lat": 29.424127,
        "lon": -98.487186,
        "walkscore": 74,
        "airbnb_count": 225,
        "other_count": 6,
        "avg_occupancy": 52.68,
        "median_price": 605000,
        "sqft_price": 369.23
    },
    "275804": {
        "name": "United Homeowners",
        "city": "San Antonio",
        "county": "Bexar",
        "state": "TX",
        "lat": 29.44015,
        "lon": -98.427296,
        "walkscore": 14,
        "airbnb_count": 210,
        "other_cou

In [72]:
with open(f'../data/mash-api/{datef}/top_nbh_overviewSPF-{datef}.json', 'w') as f1:
    json.dump(spfOVW_json,f1,indent=4)
    print("The top neighborhood specific overview json file is created")
f1.close()

The top neighborhood specific overview json file is created


In [73]:
# END JSON PARSE FROM API